### Missing Photo Detect

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

In [4]:
def word_usage_result(word_list, df_target, target_column, word_usage_min, word_usage_max): # word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max)
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        #opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value])
            #result_list_select.append([target_value,opt_value])  
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}"])
    #df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    #df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [5]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [6]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [7]:
image_path_folder = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/1000 Words/Images_Crop_Size"
path_folder = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 2/Data/Deployment"
#file = "Talk Time 2 Video List.xlsx"
#sheet = "Sheet1"  # Sheet1

In [8]:
#df_video_file = pd.read_excel(f"{path_folder}/{file}")
#df_video_file

,2Gram,3Gram,4Gram,5Gram
0,ama bu,böyle bir şey,başka bir şey yok,bu da ne demek oluyor
1,bu ne,bir kez daha,bir fikrin var mı,başka bir şey var mı
2,biraz daha,dalga mı geçiyorsun,ne var ne yok,o kadar da kötü değil
3,bu kadar,ne yazık ki,ben de seni seviyorum,daha iyi bir fikrim var
4,hazır mısın,önemli bir şey,bir şey sorabilir miyim,sana bir şey sorabilir miyim
5,ne zaman,hiçbir şey yok,bir şey mi var,yapabileceğim bir şey var mı
6,bir tane,öyle bir şey,burada ne işin var,NaN
7,ne diyorsun,bir fikrim var,ne olduğunu biliyor musun,NaN
8,ne olacak,bir şey daha,önemli bir şey değil,NaN
9,bir gün,bir şey mi,ben de öyle düşünmüştüm,NaN


In [ ]:
ngram_video_file_list = glob.glob(f"{path_folder}/Talk Time 2 * Video List.xlsx")
ngram_video_file_list

In [ ]:
df_video_file = pd.DataFrame()
ngram_num = 1
for file in ngram_video_file_list:
    df_var = pd.read_excel(f"{file}")
    df_var = df_var.iloc[:,[0]]
    old_name = df_var.columns[0]
    df_var = df_var.rename(columns={f"{old_name}":f"ngram{ngram_num}"})
    df_video_file = pd.concat([df_video_file,df_var], axis=1)
    ngram_num +=1 
df_video_file

In [9]:
df_file_count = word_count_result(df_video_file,df_video_file.columns)
df_file_count

,word,word_count
0,bir,25
1,ne,15
2,şey,14
3,bu,9
4,daha,8
...,...,...
74,kez,1
75,ki,1
76,kötü,1
77,misin,1


In [10]:
image_file_list = os.listdir(image_path_folder)
filename_without_ext_list = []
for file in image_file_list:
    # file and extention
    file_without_ext = os.path.splitext(file)[0]
    filename_without_ext_list.append(file_without_ext)
filename_without_ext_list

['ama',
 'artık',
 'asla',
 'aslında',
 'aynı',
 'baba',
 'bak',
 'bakalım',
 'bana',
 'belki',
 'ben',
 'beni',
 'benimle',
 'bile',
 'biliyorsun',
 'biliyorum',
 'diye',
 'acele',
 'biri',
 'biz',
 'bize',
 'bizi',
 'bizim',
 'bu',
 'buna',
 'bunu',
 'burada',
 'buraya',
 'bütün',
 'gerçek',
 'böyle',
 'çok',
 'de',
 'doğru',
 'dostum',
 'dur',
 'ederim',
 'efendim',
 'et',
 'evet',
 'fazla',
 'gece',
 'gel',
 'geldi',
 'geliyor',
 'gerçekten',
 'geri',
 'gibi',
 'misin',
 'ana',
 'göre',
 'gün',
 'güzel',
 'hadi',
 'hala',
 'harika',
 'haydi',
 'her',
 'herkes',
 'hey',
 'hiçbir',
 'için',
 'ile',
 'ilk',
 'iş',
 'istiyorsun',
 'istiyorum',
 'iyi',
 'izin',
 'kadar',
 'mısın',
 'işte',
 'ki',
 'kız',
 'lazım',
 'lütfen',
 'merhaba',
 'musun',
 'anda',
 'basit',
 'nerede',
 'o',
 'olabilir',
 'olacak',
 'olan',
 'olarak',
 'oldu',
 'olduğunu',
 'olmak',
 'olmaz',
 'olsun',
 'olur',
 'oluyor',
 'ona',
 'önce',
 'onları',
 'onu',
 'onunla',
 'orada',
 'öyle',
 'pekala',
 'sadece',
 'sa

In [11]:
set_image_file = set(filename_without_ext_list)
set_video_file = set(df_file_count["word"])

In [12]:
set_video_file.difference(set_image_file)

{'dalga',
 'dersin',
 'diyorsun',
 'duyuyor',
 'düşünmüştüm',
 'fikir',
 'fikrin',
 'geçiyorsun',
 'hazır',
 'hepsi',
 'kez',
 'olursa',
 'sakin',
 'saniye',
 'seviyorum',
 'sizin',
 'sorabilir',
 'tane',
 'yapabileceğim',
 'yapabilirim',
 'yazık'}